In [11]:
import sys
import os
sys.path.append('D:\\aiprof\\app')  # Ensure this path is correct
sys.path.append('D:\\aiprof\\')

In [10]:
from typing import List, Dict, Optional
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime
import logging
import uuid

from app.utils.vector_store import vector_store
from app.modals.chat import get_llm
from models import Conversation
from users.models import Assistant, SupabaseUser

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ChatModule:
    def __init__(self):
        """Initialize the chat module with necessary components."""
        self.llm = get_llm()
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
    def _create_prompt(self, assistant_config) -> ChatPromptTemplate:
        """Create a chat prompt template based on assistant configuration."""
        template = """You are a teaching assistant specialized in {subject}.
        
        Instructions: {instructions}
        
        Context from knowledge base:
        {context}
        
        Chat History:
        {chat_history}
        
        Human Question: {question}
        
        Provide a clear, detailed response based on the context and your expertise. If the question cannot be fully answered using the provided context, state that clearly but provide the best possible answer from what is available."""
        
        prompt = ChatPromptTemplate.from_template(template)
        return prompt

    def get_relevant_context(self, query: str, ass_id: str, k: int = 3) -> str:
        """Retrieve relevant context from vector store."""
        try:
            results = vector_store.similarity_search(
                query,
                k=k,
                filter={"ass_id": ass_id}
            )
            return "\n".join([doc.page_content for doc in results])
        except Exception as e:
            logger.error(f"Error retrieving context: {e}")
            return ""

    def save_chat_history(self, user_id: str, ass_id: str, 
                         prompt: str, content: str, 
                         conversation_id: Optional[uuid.UUID] = None) -> None:
        """Save chat interaction to Django model."""
        try:
            # Get the related models instances
            user = SupabaseUser.objects.get(id=user_id)
            assistant = Assistant.objects.get(ass_id=ass_id)
            
            # If no conversation_id provided, create a new one
            if not conversation_id:
                conversation_id = uuid.uuid4()
            
            # Create new conversation entry
            Conversation.objects.create(
                users_id=user,
                ass_id=assistant,
                prompt=prompt,
                content=content,
                conversation_id=conversation_id
            )
            
            logger.info(f"Saved chat history for conversation {conversation_id}")
        except Exception as e:
            logger.error(f"Error saving chat history: {e}")
            raise

    def get_chat_history(self, ass_id: str, user_id: str, 
                        conversation_id: Optional[uuid.UUID] = None,
                        limit: int = 10) -> List[Dict]:
        """Retrieve chat history from Django model."""
        try:
            # Base query
            query = Conversation.objects.filter(
                ass_id__ass_id=ass_id,
                users_id__id=user_id
            )
            
            # Add conversation_id filter if provided
            if conversation_id:
                query = query.filter(conversation_id=conversation_id)
            
            # Get latest conversations
            conversations = query.order_by('-created_at')[:limit]
            
            # Convert to list of dicts for consistency with existing code
            return [
                {
                    'question': conv.prompt,
                    'answer': conv.content,
                    'timestamp': conv.created_at,
                    'conversation_id': conv.conversation_id
                }
                for conv in conversations
            ]
        except Exception as e:
            logger.error(f"Error retrieving chat history: {e}")
            return []

    def process_message(self, message: str, ass_id: str, 
                       user_id: str, assistant_config: dict,
                       conversation_id: Optional[uuid.UUID] = None) -> str:
        """Process a chat message and generate a response."""
        try:
            # Get relevant context
            context = self.get_relevant_context(message, ass_id)
            
            # Create prompt
            prompt = self._create_prompt(assistant_config)
            
            # Get chat history
            chat_history = self.get_chat_history(
                ass_id, 
                user_id, 
                conversation_id=conversation_id
            )
            chat_history_str = "\n".join([
                f"Human: {chat['question']}\nAssistant: {chat['answer']}"
                for chat in chat_history
            ])

            # Create chain
            chain = (
                {"context": lambda x: context,
                 "question": RunnablePassthrough(),
                 "chat_history": lambda x: chat_history_str,
                 "subject": lambda x: assistant_config["subject"],
                 "instructions": lambda x: assistant_config["teacher_instructions"]}
                | prompt
                | self.llm
                | StrOutputParser()
            )
            
            # Generate response
            response = chain.invoke(message)
            
            # Save interaction
            self.save_chat_history(
                user_id=user_id,
                ass_id=ass_id,
                prompt=message,
                content=response,
                conversation_id=conversation_id
            )
            
            return response
            
        except Exception as e:
            logger.error(f"Error processing message: {e}")
            return "I apologize, but I encountered an error processing your message. Please try again."

    def clear_chat_history(self, ass_id: str, user_id: str, 
                          conversation_id: Optional[uuid.UUID] = None) -> bool:
        """Clear chat history for a specific assistant and user."""
        try:
            query = Conversation.objects.filter(
                ass_id__ass_id=ass_id,
                users_id__id=user_id
            )
            
            if conversation_id:
                query = query.filter(conversation_id=conversation_id)
                
            query.delete()
            return True
        except Exception as e:
            logger.error(f"Error clearing chat history: {e}")
            return False
            
    def get_conversation_list(self, user_id: str) -> List[Dict]:
        """Get list of unique conversations for a user."""
        try:
            conversations = Conversation.objects.filter(
                users_id__id=user_id
            ).values(
                'conversation_id',
                'created_at'
            ).distinct().order_by('-created_at')
            
            return [
                {
                    'conversation_id': str(conv['conversation_id']),
                    'created_at': conv['created_at']
                }
                for conv in conversations
            ]
        except Exception as e:
            logger.error(f"Error retrieving conversation list: {e}")
            return []

ModuleNotFoundError: No module named 'app'